In [1]:
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

 
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [3]:
with open("/Users/pankaj/dev/git/smu/nlp337/hindi/data/godan/godan_novel.txt") as f:
    lines = f.readlines()

text = "".join([l for l in lines if l.strip()!=''])

# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])
# The maximum length sentence we want for a single input in characters
seq_length = 30
examples_per_epoch = len(text)//(seq_length+1)
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
dataset = sequences.map(split_input_target)

Length of text: 780094 characters


In [4]:
from nltk import word_tokenize

In [5]:
words = word_tokenize(text)

In [25]:
len(set(words))

13888

In [7]:
from collections import Counter
c = Counter(words)

In [8]:
import operator
x = c
sorted_x = sorted(x.items(), key=operator.itemgetter(1), reverse=True)

In [9]:
sorted_x

[(',', 7422),
 ('में', 3275),
 ('और', 3083),
 ('तो', 2982),
 ('के', 2671),
 ('से', 2566),
 ('की', 2476),
 ('है', 2149),
 ("'", 2137),
 ('को', 2005),
 ('का', 1959),
 ('नहीं', 1943),
 ('न', 1942),
 ('ने', 1837),
 ('है।', 1729),
 ('--', 1707),
 ('भी', 1691),
 ('हो', 1574),
 ('पर', 1557),
 ('वह', 1509),
 ('?', 1310),
 ('ही', 1197),
 ('कि', 1139),
 ('मैं', 1050),
 (';', 998),
 ('एक', 951),
 ('था।', 944),
 ('यह', 912),
 ('हैं', 829),
 ('था', 793),
 ('क्या', 784),
 ('कोई', 751),
 ('कर', 750),
 ('थी।', 719),
 ('कुछ', 711),
 ('कहा', 690),
 ('!', 682),
 ('होरी', 621),
 ('उसे', 621),
 ('हैं।', 612),
 ('अब', 594),
 ('जो', 592),
 ('थी', 583),
 ('घर', 582),
 ('लिए', 581),
 ('अपने', 561),
 ('इस', 545),
 ('उसके', 533),
 ('थे।', 501),
 ('रुपए', 482),
 ('किसी', 454),
 ('तुम', 451),
 ('लेकिन', 439),
 ('मालती', 438),
 ('आप', 437),
 ('उसकी', 419),
 ('थे', 417),
 ('मुझे', 416),
 ('गोबर', 414),
 ('तक', 414),
 ('साहब', 403),
 ('क्यों', 401),
 ('जब', 389),
 ('मेहता', 388),
 ('रहा', 383),
 ('रही', 381),
 ('गया'

In [10]:
len(words)

175991

In [11]:
len(vocab)

105

In [12]:
x1 = Counter(text)
sorted_x1 = sorted(x1.items(), key=operator.itemgetter(1), reverse=True)
sorted_x1

[(' ', 162154),
 ('ा', 53918),
 ('े', 38633),
 ('क', 36388),
 ('र', 33782),
 ('ह', 30792),
 ('ी', 29432),
 ('न', 27255),
 ('स', 22443),
 ('त', 21784),
 ('ो', 20147),
 ('म', 19922),
 ('ं', 16317),
 ('ि', 15854),
 ('ल', 15494),
 ('्', 14593),
 ('य', 13437),
 ('प', 12257),
 ('।', 11850),
 ('द', 11397),
 ('ब', 10746),
 ('ग', 10299),
 ('ज', 10137),
 ('ु', 9869),
 ('ै', 8520),
 ('़', 8058),
 ('व', 7854),
 (',', 7422),
 ('-', 6461),
 ('ँ', 5742),
 ('थ', 5692),
 ('च', 5661),
 ('उ', 5334),
 ('भ', 5123),
 ('आ', 5087),
 ('ख', 5004),
 ('ू', 4808),
 ('ड', 4582),
 ('अ', 4331),
 ('ए', 3282),
 ('औ', 3247),
 ('ट', 2914),
 ("'", 2668),
 ('\n', 2642),
 ('श', 2496),
 ('छ', 2481),
 ('ध', 2351),
 ('इ', 2336),
 ('ई', 1914),
 ('झ', 1912),
 ('ठ', 1541),
 ('फ', 1421),
 ('?', 1310),
 ('ौ', 1290),
 ('घ', 1122),
 ('ष', 1096),
 (';', 998),
 ('!', 682),
 ('ण', 681),
 ('ओ', 676),
 ('ढ', 589),
 ('ऊ', 510),
 ('ऐ', 477),
 ('ृ', 371),
 ('.', 177),
 ('ः', 108),
 ('ञ', 90),
 ('1', 14),
 ('2', 14),
 ('3', 11),
 ('ऋ', 7),
 (

In [13]:
Counter(text)

Counter({'1': 14,
         '.': 177,
         '\n': 2642,
         'ह': 30792,
         'ो': 20147,
         'र': 33782,
         'ी': 29432,
         'ा': 53918,
         'म': 19922,
         ' ': 162154,
         'न': 27255,
         'े': 38633,
         'द': 11397,
         'ं': 16317,
         'ब': 10746,
         'ै': 8520,
         'ल': 15494,
         'क': 36388,
         'स': 22443,
         '-': 6461,
         'प': 12257,
         'अ': 4331,
         '्': 14593,
         'त': 21784,
         'ध': 2351,
         'ि': 15854,
         'य': 13437,
         'ग': 10299,
         'ऊ': 510,
         'ख': 5004,
         'ड': 4582,
         '़': 8058,
         'भ': 5123,
         'ज': 10137,
         '।': 11850,
         'ौ': 1290,
         'ट': 2914,
         'ू': 4808,
         'ँ': 5742,
         'ठ': 1541,
         'थ': 5692,
         'उ': 5334,
         'आ': 5087,
         ',': 7422,
         'ु': 9869,
         'छ': 2481,
         'ऐ': 477,
         'झ': 1912,
         'ए': 3282,


In [14]:
# Batch size
BATCH_SIZE = 64

BATCH_SIZE = 16


# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

# The embedding dimension
#embedding_dim = 16

# Number of RNN units
#rnn_units = 32


In [15]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)
model.summary()
model.compile(optimizer='adam', loss=loss)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (16, None, 256)           26880     
_________________________________________________________________
gru (GRU)                    (16, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (16, None, 105)           107625    
Total params: 4,072,809
Trainable params: 4,072,809
Non-trainable params: 0
_________________________________________________________________


In [16]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(16, 30, 105) # (batch_size, sequence_length, vocab_size)


In [17]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
vexample_batch_loss  = loss(target_example_batch, example_batch_predictions)
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [18]:
EPOCHS=10
#EPOCHS=5
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Train for 1572 steps
Epoch 1/10
1572/1572 [==============================] - 479s 304ms/step - loss: 2.0645
Epoch 2/10
1572/1572 [==============================] - 484s 308ms/step - loss: 1.6652
Epoch 3/10
1572/1572 [==============================] - 489s 311ms/step - loss: 1.5602
Epoch 4/10
1572/1572 [==============================] - 1777s 1s/step - loss: 1.4912
Epoch 5/10
1572/1572 [==============================] - 486s 309ms/step - loss: 1.4396
Epoch 6/10
1572/1572 [==============================] - 502s 319ms/step - loss: 1.3978
Epoch 7/10
1572/1572 [==============================] - 522s 332ms/step - loss: 1.3632
Epoch 8/10
1572/1572 [==============================] - 538s 342ms/step - loss: 1.3379
Epoch 9/10
1572/1572 [==============================] - 538s 342ms/step - loss: 1.3176
Epoch 10/10
1572/1572 [==============================] - 523s 332ms/step - loss: 1.3068


In [19]:
tf.train.latest_checkpoint(checkpoint_dir)


'./training_checkpoints/ckpt_10'

In [20]:
model2 = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model2.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model2.build(tf.TensorShape([1, None]))
model2.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            26880     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 105)            107625    
Total params: 4,072,809
Trainable params: 4,072,809
Non-trainable params: 0
_________________________________________________________________


In [21]:
#print(generate_text(model, start_string=u"ROMEO: "))

print(generate_text(model2, start_string=u"होरी"))


होरी का रोब मिला। मान लो, जिसके पाँव जैसे अँजुली- लटा उठाइए। होली ला सकती हैं। लड़का बताऊँगी; पर पण्डित ओंकारनाथों से पड़े! इसमें कोई उन्हें यों जो को वह अब उसे लेकर उसका आनन्द बोला -- कल गो रंठ लगा। अब जो झुनिया के लिए लगा बैठा। राय साहब बेसवा; हाँ नहीं, अब तुम्हारी देह खोलने से ख़ोचली -- हो योरी जब देवियों के विशुल फू़ा न पड़ जायगी, तो डाँटने लगा शंसा ही उसका घर उठा ली और जमा केवल फाड़े पर के हां देखकर बोला -- तुम्हीं लोगों की जल्दी नहीं है। यही वकारत हो चूब लूँ, जलपार कीजिए, आफ़त आलेक्शन है तुम्हारा पर खेलने लगे, तो अपने ही त्याग की मार पैसा जैसे क्या कर दे? अब मालूम होती। पंचों के केसरे लिख मुए मालगह मै सह अपने तो गलाखींचता दे देता। और उसके तपवह कि राय द्वार बनके क्या संसाक़ एक हो रही थीं। उन्होंने टाँग ही तक पते थे, सोचा, साद ही दिन एक नुष्क और पत्थिल हो जायगा। अलग हैं क्या मेरे दूसरों से चारों ओर अपनी प्रेरा सर्वनाश आया, तो शायद अबकी सज्जनता विवाह अब निराश में मर्चन का अपमान ने, ख़बरदार। अब तो सूखी ही है। बेचारखा भी नहीं। अभी आप रह गया। बोला -- अब भी तुम्हारे घर आ जायगा। गोबर बड़

In [22]:
print(generate_text(model2, start_string=u"मालती"))




मालती को डाक्टर हाथ बाँधवालियाँ भी तेरी बिसिया की याद था कि मुझे तो सब यहाँ आज लोग भी दया आता था। मन में एक पतवार हट जाती थी। वह माँ-बाप हैं सज्जनों च!
'नहीं रह जाय, मैं नहीं कह रहा हूँ, वहीं घर अब अवसर पड़ने लगी।
राय साहब ने उनका चप्पल न होगा। रंग मुख में पूजा किये, जिसके द्वार पर बैठो। मैं आपको निकाल बाँध गया कपूछा -- अब मैं अपने अब तक उसके आत्म-समर्पण के मुँह में उसके दारोग़ क्यों नहीं। यह दाया बेला और परदास की लाई और टोकरी मिल जायँ। अगर उन्हें जा बोले -- ले डिर! मेरी ँ लिए मुझे जवाब दे दूँगी; अगर तुम्हारा आसना फ्र का पीतरण? सचीप ही की रहा जाऊँगा हाटा मिलबिसे छिछ जायेंगे। अगर मैं नहीं कह सकते? आप स्वयम् गढ़ेंगे। परते तो यहाँ कोई आधार नहीं ही जाता। हाँ, भी अमौक़े पर राम तो जलम्हें विश्वास न किया हो, भी तो उनसे मिल कुछ नहीं पूछता। पुनिया पड़ रही थी। उसके पास जा जी कि मालती के पाँच ज़रा भी अपने पन्द्रह साल आकर ऐसे उदार हो जाते थे, मगर से आज पाकते मैं थकानते। वह उसके साथ है, मिस्टर तंखा बोले -- अच्छा हूँ, मन्दग़ नहीं है। '
'तुम्हारी मर जाऊँगा और मुख पर गोविन्दी की प्रतिमा की को पुआल चलव

In [26]:
print(generate_text(model2, start_string=u"वह"))


वह इस भाग में मूस्स आ गये हैं? मिरज़ाजी को भी पल्ला करके कुल की बात थी कि वह उसका ग़ासन लगाकर सता लिया और अब मतई थी, वह गोईं जाने दूँगा। इसी पर है। हाँ, मोरी बुराआ नहीं तो बोली-घा भी बड़ी लज्जाही है; लेकिन मर खो; पर पीते हैं। खन्ना ने उसका घर नहीं। पूछा -- दादा, सहुनूनिय हाथ सेंका माँगा! राँड़ तो मिल जायगा। '
मालती का रो कहा बनाया है, अदालत इसी शेत में टाँग मैल नहीं।
राय साहब ने खन्ना ने जितोई कमी थी, वह उनके मुँह पर प्रसन्न नहीं है जो कोई निभूत हो गया। हराषी अपनी दूर जानती हो कि हा? क्यों को देखी। केवल महाजन ने उसके सार पर हँसते-- तो तू वहाँ पीछे की बात में होता ही, इसकी संकी-स्याली है। '
'नहीं मालिकार! '
'तुमसे विवाह करने की ज़रूरत नहीं है। उसके एक सूने पटटकादी थी, जिससे उनकी मरदूम यों का किर लेकर रखनेवाले लं आदर्श होने में पर जाप-पानी में खुलते और । गवान् मसहोगा। उनका स्वाज़ हो जाता! तू खड़ा था, इसमें उसका ही अविवाह होने लगे। होरी आँगनी में मुलाहते हुए उन्होंने-अपना खड़ा रा स्वाद हुआ, तो कल वह निर्दयी है? उसे दाँ से घर में निर पआपके ऊँचे संसार का यौवन दिशाओं पर गाय जलायेगी; जिस पर श

In [102]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [103]:
optimizer = tf.keras.optimizers.Adam()


In [104]:
@tf.function

def train_step(inp, target):
  with tf.GradientTape() as tape:
    predictions = model(inp)
    loss = tf.reduce_mean(
        tf.keras.losses.sparse_categorical_crossentropy(
            target, predictions, from_logits=True))
  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return loss

In [105]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.381718158721924
Epoch 1 Batch 100 Loss 2.4497358798980713
Epoch 1 Loss 2.3139
Time taken for 1 epoch 54.01252794265747 sec

Epoch 2 Batch 0 Loss 2.354511022567749
Epoch 2 Batch 100 Loss 2.4325413703918457
Epoch 2 Loss 2.2089
Time taken for 1 epoch 51.287450313568115 sec

Epoch 3 Batch 0 Loss 2.1450743675231934
Epoch 3 Batch 100 Loss 1.9633461236953735
Epoch 3 Loss 2.0679
Time taken for 1 epoch 52.35612607002258 sec

Epoch 4 Batch 0 Loss 1.890282392501831
Epoch 4 Batch 100 Loss 1.9342198371887207
Epoch 4 Loss 1.9022
Time taken for 1 epoch 54.84479212760925 sec

Epoch 5 Batch 0 Loss 1.8364064693450928
Epoch 5 Batch 100 Loss 1.7047028541564941
Epoch 5 Loss 1.8721
Time taken for 1 epoch 56.429481983184814 sec

Epoch 6 Batch 0 Loss 1.6121739149093628
Epoch 6 Batch 100 Loss 1.7303071022033691
Epoch 6 Loss 1.4901
Time taken for 1 epoch 49.392452239990234 sec

Epoch 7 Batch 0 Loss 1.6311205625534058
Epoch 7 Batch 100 Loss 1.4953564405441284
Epoch 7 Loss 1.3061
Time taken

Epoch 8 Batch 0 Loss 1.2487035989761353
Epoch 8 Batch 100 Loss 1.529293179512024
Epoch 8 Loss 1.5698
Time taken for 1 epoch 49.09016799926758 sec

Epoch 9 Batch 0 Loss 1.126464605331421
Epoch 9 Batch 100 Loss 1.3225820064544678
Epoch 9 Loss 1.2992
Time taken for 1 epoch 51.88028407096863 sec

Epoch 10 Batch 0 Loss 1.0157569646835327
Epoch 10 Batch 100 Loss 1.0376957654953003
Epoch 10 Loss 1.1655
Time taken for 1 epoch 51.3341600894928 sec

